In [1]:
import numpy as np
import matplotlib.pyplot as plt
import laspy

a = np.load("points.npy")

In [2]:
a = np.reshape(a, (int(a.size/3), 3), order='F')
header = laspy.LasHeader(point_format=3, version="1.2")
header.add_extra_dim(laspy.ExtraBytesParams(name="random", type=np.int32))
header.offsets = np.min(a, axis=0)
header.scales = np.array([0.1, 0.1, 0.1])
point_cloud = laspy.LasData(header)
point_cloud.x = a[:, 0]
point_cloud.y = a[:, 1]
point_cloud.z = a[:, 2]
point_cloud.random = np.random.randint(-1503, 6546, len(point_cloud.points), np.int32)
points = a
voxel_size=0.1
nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
nb_vox.astype(int)

array([20, 20,  1])

In [3]:
# Compute the non empty voxels and keep a trace of indexes that we can relate to points in order to store points later on.
# Also Sum and count the points in each voxel.
non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(((points - np.min(points, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
idx_pts_vox_sorted=np.argsort(inverse)
len(non_empty_voxel_keys) # if you need to display how many no-empty voxels you have

400

In [4]:
#Define a function that takes as input an array of points, and a voxel size expressed in meters. It returns the sampled point cloud
def grid_subsampling(points, voxel_size):

  nb_vox=np.ceil((np.max(points, axis=0) - np.min(points, axis=0))/voxel_size)
  non_empty_voxel_keys, inverse, nb_pts_per_voxel= np.unique(((points - np.min(points, axis=0)) // voxel_size).astype(int), axis=0, return_inverse=True, return_counts=True)
  idx_pts_vox_sorted=np.argsort(inverse)
  voxel_grid={}
  voxel_ids={}
  grid_barycenter,grid_candidate_center,grid_ids=[],[],[]
  last_seen=0
  ids = np.arange(points.shape[0])

  for idx,vox in enumerate(non_empty_voxel_keys):
    voxel_grid[tuple(vox)]=points[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]]
    voxel_ids[tuple(vox)]=ids[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]]
    #voxel_ids[tuple(vox)]=ids[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]]
    
    #print(points[idx_pts_vox_sorted[last_seen:last_seen+nb_pts_per_voxel[idx]]])
    grid_barycenter.append(np.mean(voxel_grid[tuple(vox)],axis=0))
    grid_candidate_center.append(voxel_grid[tuple(vox)][np.linalg.norm(voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()])
    grid_ids.append(voxel_ids[tuple(vox)][np.linalg.norm(voxel_grid[tuple(vox)]-np.mean(voxel_grid[tuple(vox)],axis=0),axis=1).argmin()])
    last_seen+=nb_pts_per_voxel[idx]
  
  grid_candidate_center=np.vstack(grid_candidate_center)
  grid_ids = np.array(grid_ids)
  grid_ids = grid_ids.reshape(len(grid_ids),1)

  return grid_candidate_center, grid_ids

In [7]:
#Execute the function, and store the results in the grid_sampled_point_cloud variable
grid_sampled_point_cloud, grid_ids = grid_subsampling(points, 0.4)
print(grid_ids)
#print(points.shape)
#print(grid_sampled_point_cloud)

[[ 164]
 [ 484]
 [ 804]
 [1123]
 [1444]
 [ 131]
 [ 452]
 [ 812]
 [1131]
 [1452]
 [ 180]
 [ 500]
 [ 819]
 [1140]
 [1459]
 [ 148]
 [ 467]
 [ 827]
 [1107]
 [1467]
 [ 195]
 [ 476]
 [ 836]
 [1155]
 [1476]]


In [6]:
np.save("file.npy",grid_ids)